In [ ]:
# Reel Hits

In [1]:
import datetime
import spotipy
import os
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
from rapidfuzz import fuzz
from dotenv import load_dotenv
from typing import Union

load_dotenv()

cred_manager = SpotifyClientCredentials(client_id=os.getenv("SPOTIFY_CLIENT_ID"),
                                        client_secret=os.getenv("SPOTIFY_CLIENT_SECRET"))
sp = spotipy.Spotify(client_credentials_manager=cred_manager,
                     requests_timeout=15, retries=3, status_forcelist=(429, 500, 502, 503, 504))

In [2]:
SOUNDTRACK_HINTS = ("soundtrack", "original motion picture", "music from the motion picture", "original score", "ost")

def candidate_queries(title: str, year: Union[int, None]):
    base = [f'album:"{title}"', f'{title} soundtrack', f'album:"{title} Original Motion Picture Soundtrack"',
            f'album:"{title} Original Score"', f'album:"{title} Music From the Motion Picture"']
    if year:
        base += [f'album:"{title}" year:{year}', f'{title} year:{year} soundtrack']
    return base

In [3]:
def album_release_year(album):
    # 'release_date' can be "YYYY" or "YYYY-MM-DD"
    rd = album.get('release_date')
    if not rd: return None
    try:
        return datetime.date.fromisoformat(rd).year
    except:
        try: return int(rd[:4])
        except: return None

In [4]:
def heuristic_score(album, movie_title, movie_year):
    name = album.get('name', '')
    # Title similarity (robust to parentheses)
    title_sim = fuzz.token_set_ratio(movie_title, name)
    # Hints
    hint_bonus = 15 if any(h in name.lower() for h in SOUNDTRACK_HINTS) else 0
    # Year proximity
    ay = album_release_year(album)
    year_bonus = 0
    if movie_year and ay:
        diff = abs(ay - movie_year)
        year_bonus = 12 if diff <= 1 else 6 if diff == 2 else 0
    return title_sim + hint_bonus + year_bonus

In [5]:
def find_best_soundtrack_album(title, year):
    tried = set()
    best, best_score = None, -1

    for q_title in [title]:
        for q in candidate_queries(q_title, year):
            if q in tried: continue
            tried.add(q)
            res = sp.search(q=q, type='album', limit=10)
            for alb in res.get('albums', {}).get('items', []):
                score = heuristic_score(alb, title, year)
                if score > best_score:
                    best, best_score = alb, score
    return best, best_score


In [6]:
def get_album_popularity_metrics(album_id: str):
    album = sp.album(album_id)
    pop_album = album.get('popularity')  # 0–100 (may be None rarely)
    tracks = []
    # Gather all tracks’ popularity
    results = sp.album_tracks(album_id, limit=50)
    items = results.get('items', [])
    while results.get('next'):
        results = sp.next(results)
        items += results.get('items', [])
    # Fetch track details in batches to get popularity
    ids = [t['id'] for t in items if t.get('id')]
    pop_list = []
    for i in range(0, len(ids), 50):
        batch = sp.tracks(ids[i:i+50]).get('tracks', [])
        pop_list.extend([t.get('popularity') for t in batch if t and t.get('popularity') is not None])
    if pop_list:
        avg_track_pop = sum(pop_list) / len(pop_list)
        sum_track_pop = sum(pop_list)
    else:
        avg_track_pop = None
        sum_track_pop = None
    return {
        'album_popularity': pop_album,
        'avg_track_popularity': avg_track_pop,
        'sum_track_popularity': sum_track_pop,
        'n_tracks': len(ids),
    }


In [7]:
def process_movie(tconst: str, title: str, year: int, revenue: float):

    album, score = find_best_soundtrack_album(title, year)
    if not album:
        return {'tconst': tconst, 'title': title, 'year': year,
                'revenue': revenue, 'spotify_album_id': None, 'match_score': None}

    metrics = get_album_popularity_metrics(album['id'])
    return {
        'tconst': tconst,
        'title': title,
        'year': year,
        'revenue': revenue,
        'spotify_album_id': album['id'],
        'spotify_album_name': album['name'],
        'spotify_album_year': album_release_year(album),
        'match_score': score,
        **metrics
    }


In [8]:
df = pd.read_csv("data\\clean_tmdb.csv")
print("Loaded clean_tmdb.csv with shape:", df.shape)

df = df[(df['revenue'] > 0) & (df['release_date'].notna()) & (df['imdb_id'].notna()) & (df['title'].notna())]
df = df[df['release_date'].apply(lambda x: datetime.date.fromisoformat(x).year if pd.notna(x) else None) >= 1990]
df = df[df['original_language'] == 'en']
df = df[(df['vote_count'] >= 2000)]
df = df[['imdb_id', 'title', 'release_date', 'revenue']]

print("Filtered DataFrame shape:", df.shape)

results = []
tuples = list(df.itertuples(index=False))
from tqdm import tqdm
for row in tqdm(tuples[:500]):
    tconst, title, year, revenue = row.imdb_id, row.title, datetime.date.fromisoformat(row.release_date).year, float(row.revenue)
    result = process_movie(tconst, title, year, revenue)
    results.append(result)

results_df = pd.DataFrame(results)
print(results_df.head())
results_df.to_csv("data\\reel_hits.csv", index=False)

Loaded clean_tmdb.csv with shape: (16013, 24)
Filtered DataFrame shape: (745, 4)


100%|██████████| 500/500 [18:25<00:00,  2.21s/it]

      tconst            title  year       revenue        spotify_album_id  \
0  tt1375666        Inception  2010  8.255328e+08  2qvA7HmSg1iM6XMiFF76dp   
1  tt0816692     Interstellar  2014  7.017292e+08  3B61kSKTxlY36cYgzvf3cP   
2  tt0468569  The Dark Knight  2008  1.004558e+09  63uFfOZpC7jrV7wfuBY2lX   
3  tt0499549           Avatar  2009  2.923706e+09  2ca6dgorijp5LW4vuZCLJm   
4  tt0848228     The Avengers  2012  1.518816e+09  3wvpIkfl4oOgZLMaQBWadf   

                                  spotify_album_name  spotify_album_year  \
0          Inception (Music from the Motion Picture)                2010   
1  Interstellar (Original Motion Picture Soundtra...                2014   
2  The Dark Knight (Original Motion Picture Sound...                2008   
3                   Avatar (Soundtrack for Trailers)                2018   
4  The Avengers (Original Motion Picture Soundtrack)                2012   

   match_score  album_popularity  avg_track_popularity  sum_track_popularity  \


In [123]:
metrics = get_album_popularity_metrics("37ddKI6C7HW9O1gX1gI0ei")

print(metrics)

{'album_popularity': 22, 'avg_track_popularity': 14.4, 'sum_track_popularity': 144, 'n_tracks': 10}


In [119]:
row = tuples[539]
tconst, title, year, revenue = row.imdb_id, row.title, datetime.date.fromisoformat(row.release_date).year, float(row.revenue)
result = process_movie(tconst, title, year, revenue)
print(result)

{'tconst': 'tt1398426', 'title': 'Straight Outta Compton', 'year': 2015, 'revenue': 201634991.0, 'spotify_album_id': '1rMnLDmzyEBRiCj7yoGK2n', 'spotify_album_name': 'Straight Outta Compton (Music From The Motion Picture)', 'spotify_album_year': 2016, 'match_score': 127.0, 'album_popularity': 41, 'avg_track_popularity': 23.705882352941178, 'sum_track_popularity': 403, 'n_tracks': 17}
